In [1]:
# -*- coding: utf-8 -*-

import argparse
import shutil

import numpy as np
import torch
from torch.autograd import Variable, grad
from torch.nn.init import xavier_normal
from torchvision import datasets, transforms
import torchvision.utils as vutils

from data import *
from models import *
from util import *
import time

In [2]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


def adjust_learning_rate(optimizer, epoch, g):
    """Sets the learning rate to the initial LR decayed by 10 every 10 epochs"""
    if g == 'g':
        lr = learning_rate * (0.5 ** (epoch // 20))
    else:
        lr = learning_rate * (0.5 ** (epoch // 20))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [3]:
print('Preparing dataset A...')
A_train_dataset = AttrDataset('./extra_data/reduced_hair.csv', DATA_PATH, transform=face_transform)
#A_train_dataset = AttrDataset('./face_hair_tags.csv', DATA_PATH, transform=face_transform)
train_loader = DataLoader(A_train_dataset, batch_size=batch_size, num_workers=8, shuffle=True, drop_last=True)

Preparing dataset A...


In [4]:
generator = Generator()
generator.apply(weights_init)

discriminator = Discriminator2()
discriminator.apply(weights_init)

opt_g = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
opt_d = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

In [5]:
# resume_file = './models_big/Epoch:_049.pt'

# if resume_file:
#     if os.path.isfile(resume_file):
#         print("=> loading checkpoint '{}'".format(resume_file))
#         checkpoint = torch.load(resume_file)
#         start_epoch = checkpoint['epoch'] + 1
#         #best_loss = checkpoint['loss_g'] + checkpoint['loss_d']
#         generator.load_state_dict(checkpoint['g_state_dict'])
#         discriminator.load_state_dict(checkpoint['d_state_dict'])
#         #opt_g.load_state_dict(checkpoint['g_optimizer'])
#         #opt_d.load_state_dict(checkpoint['d_optimizer'])
#         print("=> loaded checkpoint '{}' (epoch {})"
#                .format(resume_file, checkpoint['epoch']))
#     else:
#         print("=> no checkpoint found at '{}'".format(resume_file))

In [6]:
start_epoch

0

In [7]:
criterion = torch.nn.BCEWithLogitsLoss() # StableBCELoss() #
X = Variable(torch.FloatTensor(batch_size, 3, imsize, imsize))
z = Variable(torch.FloatTensor(batch_size, z_dim))
tags = Variable(torch.FloatTensor(batch_size, tag_num))
labels = Variable(torch.FloatTensor(batch_size))
tag = torch.LongTensor(batch_size)
if cuda:
    generator.cuda()
    discriminator.cuda()
    criterion.cuda()
    X, z, tags, labels,tag = X.cuda(), z.cuda(), tags.cuda(), labels.cuda(),tag.cuda()
    

In [8]:
from torchsummary import summary
summary(generator,(1,128+4))
#summary(discriminator,(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 1, 16384]       2,179,072
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
            Conv2d-4           [-1, 64, 16, 16]          36,928
       BatchNorm2d-5           [-1, 64, 16, 16]             128
              ReLU-6           [-1, 64, 16, 16]               0
            Conv2d-7           [-1, 64, 16, 16]          36,928
       BatchNorm2d-8           [-1, 64, 16, 16]             128
     residualBlock-9           [-1, 64, 16, 16]               0
           Conv2d-10           [-1, 64, 16, 16]          36,928
      BatchNorm2d-11           [-1, 64, 16, 16]             128
             ReLU-12           [-1, 64, 16, 16]               0
           Conv2d-13           [-1, 64, 16, 16]          36,928
      BatchNorm2d-14           [-1, 64,

In [ ]:
allone = [1.0 for i in range(32)]
allzero =[0.0 for i in range(32)]
def edit(tar,allone,allzero):
    out = []
    for i in tar:
        tmp = []
        for j in range(4):
            if j == i:
                tmp += allzero
            else:
                tmp += allone
        out.append(tmp)
    out = torch.FloatTensor(out)
    return out.cuda()

In [ ]:
# Train
best_loss = float('inf')
#embedding = nn.Embedding(batch_size,128).cuda()
k = 1
max_epochs += start_epoch
for epoch in range(start_epoch, max_epochs):

    adjust_learning_rate(opt_g, epoch,'g')
    adjust_learning_rate(opt_d, epoch,'d')

    for batch_idx, (data, target) in enumerate(train_loader):
        if k%5 == 0:
            k = 0
        if epoch > 20:
            k += 1
        
        X.data.copy_(data)

        # Update discriminator
        # train with real
        tags.data.copy_(target)
        discriminator.zero_grad()
        pred_real, pred_real_tag = discriminator(X)
        labels.data.fill_(1.0)
        loss_d_real_label = criterion(torch.squeeze(pred_real), labels)
        loss_d_real_tag = criterion(pred_real_tag, tags)
        loss_d_real = lambda_adv * loss_d_real_label + loss_d_real_tag
        #if (k-1)%5 == 0:
        loss_d_real.backward()

        # train with fake
        z.data.normal_(0, 1)
        np_tar = np.argmax(target.numpy(),axis = 1)
        tag = torch.LongTensor(np_tar).cuda()
        #embed = embedding(tag).view(batch_size,-1)
        #xx = z.mul(embed)
        
        reduced = edit(np_tar,allone,allzero)
        x = z.mul(reduced)
        
        #tags.data.uniform_(to=1) # Continuous
        # tags.data.fill_(0.5)           # Discrete binary string
        # tags.data.bernoulli_()
        rep = torch.cat((x, tags.clone()), 1)
        fake = generator.forward(rep).detach()
        pred_fake, pred_fake_tag = discriminator(fake)
        labels.data.fill_(0.0)
        loss_d_fake_label = criterion(torch.squeeze(pred_fake), labels)
        loss_d_fake_tag = criterion(pred_fake_tag, tags)
        loss_d_fake = lambda_adv * loss_d_fake_label + loss_d_fake_tag
        #if (k-1)%5 == 0:
        loss_d_fake.backward()

        # gradient penalty
        shape = [batch_size] + [1]*(X.dim()-1)
        alpha = torch.rand(*shape)
        beta = torch.rand(X.size())
        if cuda:
            alpha = alpha.cuda()
            beta = beta.cuda()
        x_hat = Variable(alpha * X.data + (1 - alpha) * (X.data + 0.5 * X.data.std() * beta), requires_grad=True)
        pred_hat, _ = discriminator(x_hat)
        grad_out = torch.ones(pred_hat.size())
        if cuda:
            grad_out = grad_out.cuda()
        gradients = grad(outputs=pred_hat, inputs=x_hat, grad_outputs=grad_out,
                create_graph=True, retain_graph=True, only_inputs=True)[0]
        gradient_penalty = lambda_gp * ((gradients.norm(2, dim=1) - 1) ** 2).mean()
        #if (k-1)%5 == 0:
        gradient_penalty.backward()

        loss_d = loss_d_real + loss_d_fake + gradient_penalty
        opt_d.step()

        # Update generator
        generator.zero_grad()
        z.data.normal_(0, 1)
        np_tar = np.argmax(target.numpy(),axis = 1)
        tag = torch.LongTensor(np_tar).cuda()
        #embed = embedding(tag).view(batch_size,-1)
        #xx = z.mul(embed)
        
        reduced = edit(np_tar,allone,allzero)
        x = z.mul(reduced)
        
        
        #tags.data.uniform_(to=1) # Continuous
        # tags.data.fill_(0.5)           # Discrete binary string
        # tags.data.bernoulli_()
        rep = torch.cat((x, tags.clone()), 1)
        gen = generator(rep)
        pred_gen, pred_gen_tag = discriminator(gen)
        labels.data.fill_(1)

        loss_g_label = criterion(torch.squeeze(pred_gen), labels)

        loss_g_tag = criterion(pred_gen_tag, tags)
        loss_g = lambda_adv * loss_g_label + loss_g_tag
        loss_g.backward()
        opt_g.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f Loss_D_Label: %.4f Loss_G_Label: %.4f Loss_D_Tag: %.4f Loss_G_Tag: %.4f'
              % (epoch, max_epochs, batch_idx, len(train_loader),
                 loss_d.item(), loss_g.item(), loss_d_real_label.item()+loss_d_fake_label.item(),
                 loss_g_label.item(), loss_d_real_tag.item()+loss_d_fake_tag.item(),
                 loss_g_tag.item()))

        if batch_idx % 100 == 0:
            vutils.save_image(data,
                    'samples/real_samples_epoch_%03d.png' % epoch)
            fake = generator(rep)
            vutils.save_image(fake.data.view(batch_size, 3, imsize, imsize),
                    'samples/fake_samples_epoch_%03d.png' % epoch)

            is_best = False

            ### Should not be able to define a best model..
            # total_loss = loss_d.data[0] + loss_g.data[0]
            # if total_loss < best_loss:
            #     best_loss = total_loss
            #     is_best = True

            save_checkpoint({
                'epoch': epoch,
                'arch': 'ACGAN-SRResNet-DRAGAN',
                'g_state_dict': generator.state_dict(),
                'd_state_dict': discriminator.state_dict(),
                'loss_g': loss_g.item(),
                'loss_d': loss_d.item(),
                'g_optimizer' : opt_g.state_dict(),
                'd_optimizer' : opt_d.state_dict(),
            }, is_best, 'models/Epoch:_%03d.pt' % (epoch))

[0/50][0/243] Loss_D: 7.4224 Loss_G: 3.4578 Loss_D_Label: 1.3850 Loss_G_Label: 0.6972 Loss_D_Tag: 1.3823 Loss_G_Tag: 0.6688
[0/50][1/243] Loss_D: 7.3675 Loss_G: 3.4066 Loss_D_Label: 1.3825 Loss_G_Label: 0.6956 Loss_D_Tag: 1.3375 Loss_G_Tag: 0.6242
[0/50][2/243] Loss_D: 7.2467 Loss_G: 3.5845 Loss_D_Label: 1.3759 Loss_G_Label: 0.7556 Loss_D_Tag: 1.2430 Loss_G_Tag: 0.5623
[0/50][3/243] Loss_D: 7.1228 Loss_G: 3.0851 Loss_D_Label: 1.3740 Loss_G_Label: 0.6321 Loss_D_Tag: 1.1269 Loss_G_Tag: 0.5566
[0/50][4/243] Loss_D: 7.3455 Loss_G: 3.8606 Loss_D_Label: 1.4240 Loss_G_Label: 0.8222 Loss_D_Tag: 1.1496 Loss_G_Tag: 0.5717
[0/50][5/243] Loss_D: 7.2197 Loss_G: 3.6605 Loss_D_Label: 1.3924 Loss_G_Label: 0.7739 Loss_D_Tag: 1.1503 Loss_G_Tag: 0.5649
[0/50][6/243] Loss_D: 7.1483 Loss_G: 3.5344 Loss_D_Label: 1.3781 Loss_G_Label: 0.7433 Loss_D_Tag: 1.1360 Loss_G_Tag: 0.5610
[0/50][7/243] Loss_D: 7.1033 Loss_G: 3.7294 Loss_D_Label: 1.3679 Loss_G_Label: 0.7918 Loss_D_Tag: 1.1316 Loss_G_Tag: 0.5623
[0/50][8

[0/50][66/243] Loss_D: 6.9612 Loss_G: 4.4230 Loss_D_Label: 1.3343 Loss_G_Label: 0.9667 Loss_D_Tag: 1.1243 Loss_G_Tag: 0.5562
[0/50][67/243] Loss_D: 7.2498 Loss_G: 4.1385 Loss_D_Label: 1.4066 Loss_G_Label: 0.8951 Loss_D_Tag: 1.1234 Loss_G_Tag: 0.5580
[0/50][68/243] Loss_D: 7.4797 Loss_G: 3.6409 Loss_D_Label: 1.4602 Loss_G_Label: 0.7682 Loss_D_Tag: 1.1390 Loss_G_Tag: 0.5680
[0/50][69/243] Loss_D: 7.1509 Loss_G: 3.4579 Loss_D_Label: 1.3804 Loss_G_Label: 0.7245 Loss_D_Tag: 1.1292 Loss_G_Tag: 0.5600
[0/50][70/243] Loss_D: 7.0652 Loss_G: 3.4163 Loss_D_Label: 1.3575 Loss_G_Label: 0.7128 Loss_D_Tag: 1.1351 Loss_G_Tag: 0.5653
[0/50][71/243] Loss_D: 7.0249 Loss_G: 3.4003 Loss_D_Label: 1.3490 Loss_G_Label: 0.7100 Loss_D_Tag: 1.1290 Loss_G_Tag: 0.5605
[0/50][72/243] Loss_D: 6.9959 Loss_G: 3.2647 Loss_D_Label: 1.3407 Loss_G_Label: 0.6757 Loss_D_Tag: 1.1333 Loss_G_Tag: 0.5621
[0/50][73/243] Loss_D: 7.0828 Loss_G: 3.4989 Loss_D_Label: 1.3641 Loss_G_Label: 0.7341 Loss_D_Tag: 1.1264 Loss_G_Tag: 0.5625


[0/50][132/243] Loss_D: 6.9636 Loss_G: 3.6385 Loss_D_Label: 1.3346 Loss_G_Label: 0.7697 Loss_D_Tag: 1.1253 Loss_G_Tag: 0.5595
[0/50][133/243] Loss_D: 6.9632 Loss_G: 4.3451 Loss_D_Label: 1.3350 Loss_G_Label: 0.9460 Loss_D_Tag: 1.1233 Loss_G_Tag: 0.5612
[0/50][134/243] Loss_D: 7.3528 Loss_G: 4.1642 Loss_D_Label: 1.4297 Loss_G_Label: 0.8997 Loss_D_Tag: 1.1339 Loss_G_Tag: 0.5653
[0/50][135/243] Loss_D: 6.8520 Loss_G: 2.7074 Loss_D_Label: 1.3078 Loss_G_Label: 0.5364 Loss_D_Tag: 1.1208 Loss_G_Tag: 0.5616
[0/50][136/243] Loss_D: 7.2923 Loss_G: 4.5473 Loss_D_Label: 1.4157 Loss_G_Label: 0.9961 Loss_D_Tag: 1.1297 Loss_G_Tag: 0.5631
[0/50][137/243] Loss_D: 7.9226 Loss_G: 3.3403 Loss_D_Label: 1.5726 Loss_G_Label: 0.6938 Loss_D_Tag: 1.1325 Loss_G_Tag: 0.5652
[0/50][138/243] Loss_D: 7.1461 Loss_G: 3.5034 Loss_D_Label: 1.3800 Loss_G_Label: 0.7350 Loss_D_Tag: 1.1263 Loss_G_Tag: 0.5633
[0/50][139/243] Loss_D: 7.1146 Loss_G: 3.6144 Loss_D_Label: 1.3704 Loss_G_Label: 0.7629 Loss_D_Tag: 1.1329 Loss_G_Tag:

[0/50][198/243] Loss_D: 8.2281 Loss_G: 3.4037 Loss_D_Label: 1.6500 Loss_G_Label: 0.7045 Loss_D_Tag: 1.1284 Loss_G_Tag: 0.5858
[0/50][199/243] Loss_D: 6.9595 Loss_G: 3.4185 Loss_D_Label: 1.3221 Loss_G_Label: 0.7106 Loss_D_Tag: 1.1712 Loss_G_Tag: 0.5762
[0/50][200/243] Loss_D: 7.0410 Loss_G: 3.4187 Loss_D_Label: 1.3464 Loss_G_Label: 0.7133 Loss_D_Tag: 1.1553 Loss_G_Tag: 0.5656
[0/50][201/243] Loss_D: 6.9380 Loss_G: 3.5299 Loss_D_Label: 1.3289 Loss_G_Label: 0.7407 Loss_D_Tag: 1.1224 Loss_G_Tag: 0.5671
[0/50][202/243] Loss_D: 7.0414 Loss_G: 3.5267 Loss_D_Label: 1.3534 Loss_G_Label: 0.7408 Loss_D_Tag: 1.1282 Loss_G_Tag: 0.5635
[0/50][203/243] Loss_D: 7.0203 Loss_G: 3.3849 Loss_D_Label: 1.3533 Loss_G_Label: 0.7071 Loss_D_Tag: 1.1074 Loss_G_Tag: 0.5565
[0/50][204/243] Loss_D: 6.9948 Loss_G: 3.6502 Loss_D_Label: 1.3478 Loss_G_Label: 0.7748 Loss_D_Tag: 1.1040 Loss_G_Tag: 0.5509
[0/50][205/243] Loss_D: 6.7173 Loss_G: 4.2793 Loss_D_Label: 1.2725 Loss_G_Label: 0.9268 Loss_D_Tag: 1.1276 Loss_G_Tag:

[1/50][21/243] Loss_D: 7.2475 Loss_G: 3.7822 Loss_D_Label: 1.3778 Loss_G_Label: 0.8036 Loss_D_Tag: 1.2365 Loss_G_Tag: 0.5680
[1/50][22/243] Loss_D: 7.0763 Loss_G: 3.7054 Loss_D_Label: 1.3586 Loss_G_Label: 0.7810 Loss_D_Tag: 1.1419 Loss_G_Tag: 0.5815
[1/50][23/243] Loss_D: 7.0396 Loss_G: 3.6028 Loss_D_Label: 1.3513 Loss_G_Label: 0.7593 Loss_D_Tag: 1.1344 Loss_G_Tag: 0.5654
[1/50][24/243] Loss_D: 6.9726 Loss_G: 3.7056 Loss_D_Label: 1.3362 Loss_G_Label: 0.7866 Loss_D_Tag: 1.1278 Loss_G_Tag: 0.5593
[1/50][25/243] Loss_D: 6.9132 Loss_G: 3.9008 Loss_D_Label: 1.3284 Loss_G_Label: 0.8397 Loss_D_Tag: 1.0997 Loss_G_Tag: 0.5421
[1/50][26/243] Loss_D: 6.7982 Loss_G: 3.9331 Loss_D_Label: 1.3059 Loss_G_Label: 0.8424 Loss_D_Tag: 1.0749 Loss_G_Tag: 0.5635
[1/50][27/243] Loss_D: 6.6725 Loss_G: 4.2953 Loss_D_Label: 1.2772 Loss_G_Label: 0.9377 Loss_D_Tag: 1.0640 Loss_G_Tag: 0.5445
[1/50][28/243] Loss_D: 6.9063 Loss_G: 3.6632 Loss_D_Label: 1.3402 Loss_G_Label: 0.7792 Loss_D_Tag: 1.0458 Loss_G_Tag: 0.5464


...

[2/50][41/243] Loss_D: 7.9144 Loss_G: 2.9979 Loss_D_Label: 1.6266 Loss_G_Label: 0.6397 Loss_D_Tag: 0.9091 Loss_G_Tag: 0.4390
[2/50][42/243] Loss_D: 6.5648 Loss_G: 3.2893 Loss_D_Label: 1.2993 Loss_G_Label: 0.7089 Loss_D_Tag: 0.8690 Loss_G_Tag: 0.4537
[2/50][43/243] Loss_D: 7.3598 Loss_G: 3.7787 Loss_D_Label: 1.4974 Loss_G_Label: 0.8382 Loss_D_Tag: 0.8713 Loss_G_Tag: 0.4261
[2/50][44/243] Loss_D: 6.9510 Loss_G: 4.0615 Loss_D_Label: 1.3796 Loss_G_Label: 0.9019 Loss_D_Tag: 0.9336 Loss_G_Tag: 0.4538
[2/50][45/243] Loss_D: 6.8058 Loss_G: 4.1186 Loss_D_Label: 1.3592 Loss_G_Label: 0.9166 Loss_D_Tag: 0.8695 Loss_G_Tag: 0.4521
[2/50][46/243] Loss_D: 6.5706 Loss_G: 4.6450 Loss_D_Label: 1.3069 Loss_G_Label: 1.0572 Loss_D_Tag: 0.8436 Loss_G_Tag: 0.4164
[2/50][47/243] Loss_D: 6.6323 Loss_G: 5.1582 Loss_D_Label: 1.3204 Loss_G_Label: 1.1831 Loss_D_Tag: 0.8514 Loss_G_Tag: 0.4257
[2/50][48/243] Loss_D: 6.6389 Loss_G: 4.7939 Loss_D_Label: 1.3022 Loss_G_Label: 1.0922 Loss_D_Tag: 0.9308 Loss_G_Tag: 0.4253


[2/50][107/243] Loss_D: 5.6408 Loss_G: 5.8035 Loss_D_Label: 1.0976 Loss_G_Label: 1.3806 Loss_D_Tag: 0.7519 Loss_G_Tag: 0.2812
[2/50][108/243] Loss_D: 6.8092 Loss_G: 3.6480 Loss_D_Label: 1.3752 Loss_G_Label: 0.8334 Loss_D_Tag: 0.8100 Loss_G_Tag: 0.3144
[2/50][109/243] Loss_D: 5.6879 Loss_G: 5.7585 Loss_D_Label: 1.1324 Loss_G_Label: 1.3652 Loss_D_Tag: 0.6595 Loss_G_Tag: 0.2976
[2/50][110/243] Loss_D: 5.8482 Loss_G: 3.9522 Loss_D_Label: 1.1437 Loss_G_Label: 0.9044 Loss_D_Tag: 0.7748 Loss_G_Tag: 0.3346
[2/50][111/243] Loss_D: 5.3375 Loss_G: 5.7470 Loss_D_Label: 1.0211 Loss_G_Label: 1.3574 Loss_D_Tag: 0.7543 Loss_G_Tag: 0.3173
[2/50][112/243] Loss_D: 4.6320 Loss_G: 6.9978 Loss_D_Label: 0.8699 Loss_G_Label: 1.6786 Loss_D_Tag: 0.6542 Loss_G_Tag: 0.2834
[2/50][113/243] Loss_D: 4.8321 Loss_G: 5.3638 Loss_D_Label: 0.8878 Loss_G_Label: 1.2626 Loss_D_Tag: 0.7836 Loss_G_Tag: 0.3134
[2/50][114/243] Loss_D: 7.1919 Loss_G: 27.3297 Loss_D_Label: 1.4687 Loss_G_Label: 6.7543 Loss_D_Tag: 0.8197 Loss_G_Tag

[2/50][172/243] Loss_D: 7.0122 Loss_G: 5.2125 Loss_D_Label: 1.4521 Loss_G_Label: 1.2449 Loss_D_Tag: 0.7047 Loss_G_Tag: 0.2330
[2/50][173/243] Loss_D: 7.0963 Loss_G: 4.7323 Loss_D_Label: 1.5115 Loss_G_Label: 1.1412 Loss_D_Tag: 0.5509 Loss_G_Tag: 0.1676
[2/50][174/243] Loss_D: 6.2801 Loss_G: 4.0184 Loss_D_Label: 1.3070 Loss_G_Label: 0.9555 Loss_D_Tag: 0.5534 Loss_G_Tag: 0.1964
[2/50][175/243] Loss_D: 4.4901 Loss_G: 3.4043 Loss_D_Label: 0.8384 Loss_G_Label: 0.7852 Loss_D_Tag: 0.6391 Loss_G_Tag: 0.2636
[2/50][176/243] Loss_D: 5.0410 Loss_G: 4.9073 Loss_D_Label: 0.9700 Loss_G_Label: 1.1751 Loss_D_Tag: 0.6630 Loss_G_Tag: 0.2070
[2/50][177/243] Loss_D: 6.2006 Loss_G: 4.9257 Loss_D_Label: 1.2861 Loss_G_Label: 1.1846 Loss_D_Tag: 0.5575 Loss_G_Tag: 0.1872
[2/50][178/243] Loss_D: 5.7714 Loss_G: 4.2133 Loss_D_Label: 1.1705 Loss_G_Label: 0.9944 Loss_D_Tag: 0.5911 Loss_G_Tag: 0.2359
[2/50][179/243] Loss_D: 4.5530 Loss_G: 4.1675 Loss_D_Label: 0.8332 Loss_G_Label: 0.9871 Loss_D_Tag: 0.7231 Loss_G_Tag:

[2/50][238/243] Loss_D: 6.1155 Loss_G: 4.6989 Loss_D_Label: 1.2570 Loss_G_Label: 1.1197 Loss_D_Tag: 0.5891 Loss_G_Tag: 0.2203
[2/50][239/243] Loss_D: 4.7645 Loss_G: 2.3955 Loss_D_Label: 0.9215 Loss_G_Label: 0.5450 Loss_D_Tag: 0.5810 Loss_G_Tag: 0.2154
[2/50][240/243] Loss_D: 7.7370 Loss_G: 3.9758 Loss_D_Label: 1.6490 Loss_G_Label: 0.9544 Loss_D_Tag: 0.6422 Loss_G_Tag: 0.1582
[2/50][241/243] Loss_D: 5.8070 Loss_G: 4.9132 Loss_D_Label: 1.1842 Loss_G_Label: 1.1911 Loss_D_Tag: 0.5718 Loss_G_Tag: 0.1488
[2/50][242/243] Loss_D: 6.0434 Loss_G: 4.1630 Loss_D_Label: 1.2580 Loss_G_Label: 1.0042 Loss_D_Tag: 0.5128 Loss_G_Tag: 0.1460
[3/50][0/243] Loss_D: 5.5410 Loss_G: 4.3059 Loss_D_Label: 1.1566 Loss_G_Label: 1.0425 Loss_D_Tag: 0.4162 Loss_G_Tag: 0.1359
[3/50][1/243] Loss_D: 5.9532 Loss_G: 3.5621 Loss_D_Label: 1.2527 Loss_G_Label: 0.8581 Loss_D_Tag: 0.4439 Loss_G_Tag: 0.1298
[3/50][2/243] Loss_D: 6.5002 Loss_G: 4.0225 Loss_D_Label: 1.3342 Loss_G_Label: 0.9639 Loss_D_Tag: 0.6650 Loss_G_Tag: 0.166

[3/50][61/243] Loss_D: 5.9516 Loss_G: 4.2131 Loss_D_Label: 1.2046 Loss_G_Label: 1.0079 Loss_D_Tag: 0.6352 Loss_G_Tag: 0.1813
[3/50][62/243] Loss_D: 5.8625 Loss_G: 4.6077 Loss_D_Label: 1.1930 Loss_G_Label: 1.1020 Loss_D_Tag: 0.5927 Loss_G_Tag: 0.1995
[3/50][63/243] Loss_D: 5.9988 Loss_G: 4.9016 Loss_D_Label: 1.2548 Loss_G_Label: 1.1969 Loss_D_Tag: 0.4818 Loss_G_Tag: 0.1139
[3/50][64/243] Loss_D: 5.9550 Loss_G: 4.6040 Loss_D_Label: 1.2528 Loss_G_Label: 1.1106 Loss_D_Tag: 0.4455 Loss_G_Tag: 0.1617
[3/50][65/243] Loss_D: 5.6789 Loss_G: 4.6333 Loss_D_Label: 1.1799 Loss_G_Label: 1.1268 Loss_D_Tag: 0.4617 Loss_G_Tag: 0.1262
[3/50][66/243] Loss_D: 5.2673 Loss_G: 4.2793 Loss_D_Label: 1.0922 Loss_G_Label: 1.0413 Loss_D_Tag: 0.4008 Loss_G_Tag: 0.1141
[3/50][67/243] Loss_D: 5.6700 Loss_G: 4.6035 Loss_D_Label: 1.1555 Loss_G_Label: 1.1226 Loss_D_Tag: 0.5506 Loss_G_Tag: 0.1129
[3/50][68/243] Loss_D: 5.4668 Loss_G: 4.7367 Loss_D_Label: 1.1281 Loss_G_Label: 1.1381 Loss_D_Tag: 0.4573 Loss_G_Tag: 0.1841
